In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display
from openai import OpenAI
import locale
import numpy as np
from sklearn.metrics import accuracy_score

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Load environment variables from .env
load_dotenv()

In [ ]:
# Create a SQLAlchemy engine
db_url = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT', 15432)}/{os.getenv('DB_NAME')}"
engine = create_engine(db_url)

# Database Pulls

## Categories

In [ ]:
# Query the database
query = """
select
    parents.id as parent_id,
    parents.name as parent_name,
    categories.id as category_id,
    categories.name as category_name
from categories
left join categories as parents on coalesce(categories.parent_id, categories.id) = parents.id
order by parents.id, categories.id
"""
df_cat = pd.read_sql(query, engine)

# Display the DataFrame
print(df_cat)

## Transactions

In [ ]:
# Query the database
query = """
select transactions.id,
       transaction_date,
       description,
       amount,
       categories.name as category_name
from transactions
    left join categories on transactions.category_id = categories.id
where account_id = 2 and amount < 0
order by transaction_date desc, description, id
limit 10
"""
df_trans = pd.read_sql(query, engine)
df_trans

In [ ]:
transaction = df_trans.iloc[0]
transaction

# OpenAI API Call

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
openai = OpenAI()

## System prompt

In [ ]:
def create_system_prompt(input_df):
    categories_markdown = input_df.loc[:, ['parent_name', 'category_name']].to_markdown(index=False)

    system_prompt = (f"You are labeling the categories of transactions for a personal budget. "
                     f"The following categories are available for you:\n\n{categories_markdown}\n\n"
                     "Reply only with the category name, no explanation")
    return system_prompt

system_prompt = create_system_prompt(df_cat)
display(Markdown(create_system_prompt(df_cat)))

## Message Function

In [ ]:
def create_messages(system_prompt, transaction_description, amount):
    amount_fmt = locale.currency(amount, grouping=True)
    user_prompt = f"The transaction description is \"{transaction_description}\" and the amount is {amount_fmt}"

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "The category is:"},
    ]

system_prompt = create_system_prompt(df_cat)
messages = create_messages(system_prompt[0:500], transaction['description'], transaction['amount'])
messages

## Send to OpenAI

In [ ]:
def get_category(transaction):
    system_prompt = create_system_prompt(df_cat)

    messages = create_messages(system_prompt, transaction['description'], transaction['amount'])

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        seed=69,
        # max_tokens=5,
    )
    reply = response.choices[0].message.content

    return reply

get_category(transaction)

# Testing

In [ ]:
df_trans.loc[:, 'category_name_pred'] = df_trans.apply(get_category, axis=1)
df_trans

## Error Analysis

In [ ]:
accuracy_score(df_trans.category_name, df_trans.category_name_pred)

In [ ]:
df_trans.groupby(['category_name', 'category_name_pred']).count()